[View in Colaboratory](https://colab.research.google.com/github/robmoore/diabetes-prediction/blob/master/Create_CCS_SQLite_Database.ipynb)

In [0]:
!apt install sqlite3 -q

In [0]:
csvfile = 'dxref_2015.csv'
!curl -s 'http://www.nber.org/ahrq/ccs/2017/20170421/%24dxref%202015.csv' | tail -n +2 | sed "s/'/\"/g" > {csvfile}

In [0]:
import pandas as pd
import sqlite3
import numpy as np
  
from contextlib import closing

# Data uploaded to Google Drive below. Assumes the existence of a folder named 'Diabetes-Prediction' in your Google Drive account.
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!head {csvfile}

In [0]:
db_name = 'dxref_2015.db'
table_name = 'dxref'

df = pd.read_csv(csvfile)
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('-', '_')
df = df.applymap(lambda x: x.strip() if type(x) is str else x)
df.replace('', np.NaN)  
      
with closing(sqlite3.connect('file:{}'.format(db_name), uri=True)) as conn:    
  df.to_sql(table_name, conn, if_exists='replace', index=False)

In [0]:
df.head()

In [0]:
groups = df.groupby(['CCS_CATEGORY'])['ICD_9_CM_CODE'].apply(list)
groups.loc[49]

In [0]:
# See also: Screen DM -> V771, FHX -> V180, NEUROPATHY IN DIABETES -> 3572
with closing(sqlite3.connect('file:{}?mode=ro'.format(db_name), uri=True)) as conn:    
    dxref = pd.read_sql_query("""
    SELECT CCS_CATEGORY, ICD_9_CM_CODE
    FROM {}
    WHERE CCS_CATEGORY IN ('49', '50', '186')
    GROUP BY ICD_9_CM_CODE
    ORDER BY CCS_CATEGORY
""".format(table_name), conn)
dxref.head()

In [0]:
!rm {csvfile}
!cp {db_name} /content/gdrive/My\ Drive/Diabetes-Prediction/